In [38]:
from openai import OpenAI
import os
import ast
import re
import json
import markdown
from bs4 import BeautifulSoup
import pickle


import requests
from xml.etree import ElementTree
import ast

from collections import defaultdict
from difflib import SequenceMatcher
import numpy as np
from scipy.cluster.hierarchy import fcluster
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from linkml_runtime.loaders.yaml_loader import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView

In [57]:
openai.api_key = ""

In [58]:
# Initialize the OpenAI client
client = OpenAI(api_key=openai.api_key)#, base_url="https://api.cborg.lbl.gov")

In [64]:
# Define the function to query the OpenAI API
def query_openai(model, role, prompt):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": role, "content": prompt}
            ]
        )
        return response#.choices[0].message['content'] if response.choices else None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [8]:
schema_path = "../src/data_sheets_schema/schema/data_sheets_schema.yaml"
# Load schema
#yaml_loader = YAMLLoader()
schema_view = SchemaView(schema_path)

print(f"Schema name: {schema_view.schema.name}")
classes = schema_view.all_classes()
print(f"Classes in the schema: {list(classes.keys())}")

INFO:root:Importing standards_organization_schema as standards_organization_schema from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=../src/data_sheets_schema/schema
INFO:root:Importing standards_schema as standards_schema from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=../src/data_sheets_schema/schema
INFO:root:Importing linkml:types as /Users/marcin/Documents/VIMSS/ontology/bridge2ai/data-sheets-schema/venv/lib/python3.10/site-packages/linkml_runtime/linkml_model/model/schema/types from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=None


Schema name: data-sheets-schema
Classes in the schema: ['NamedThing', 'AnatomicalEntity', 'Organization', 'OrganizationContainer', 'Information', 'FormatDialect', 'Person', 'DatasetProperty', 'DatasetCollection', 'Dataset', 'DataSubset', 'Software', 'Purpose', 'Task', 'AddressingGap', 'Creator', 'FundingMechanism', 'Grantor', 'Grant', 'Instance', 'SamplingStrategy', 'MissingInfo', 'Relationships', 'Splits', 'DataAnomaly', 'ExternalResource', 'Confidentiality', 'ContentWarning', 'Subpopulation', 'Deidentification', 'SensitiveElement', 'InstanceAcquisition', 'CollectionMechanism', 'DataCollector', 'CollectionTimeframe', 'EthicalReview', 'DirectCollection', 'CollectionNotification', 'CollectionConsent', 'ConsentRevocation', 'DataProtectionImpact', 'PreprocessingStrategy', 'CleaningStrategy', 'LabelingStrategy', 'RawData', 'ExistingUse', 'UseRepository', 'OtherTask', 'FutureUseImpact', 'DiscouragedUse', 'ThirdPartySharing', 'DistributionFormat', 'DistributionDate', 'LicenseAndUseTerms', 'I

In [9]:
schema_view

SchemaView(schema=SchemaDefinition({
  'name': 'data-sheets-schema',
  'description': 'A LinkML schema for Datasheets for Datasets.',
  'title': 'data-sheets-schema',
  'see_also': ['https://bridge2ai.github.io/data-sheets-schema'],
  'id': 'https://w3id.org/bridge2ai/data-sheets-schema',
  'imports': ['linkml:types', 'standards_schema', 'standards_organization_schema'],
  'license': 'MIT',
  'prefixes': {'biolink': Prefix({'prefix_prefix': 'biolink', 'prefix_reference': 'https://w3id.org/biolink/vocab/'}),
    'csvw': Prefix({'prefix_prefix': 'csvw', 'prefix_reference': 'http://www.w3.org/ns/csvw#'}),
    'data_sheets_schema': Prefix({
      'prefix_prefix': 'data_sheets_schema',
      'prefix_reference': 'https://w3id.org/bridge2ai/data-sheets-schema/'
    }),
    'datasets': Prefix({'prefix_prefix': 'datasets', 'prefix_reference': 'https://w3id.org/linkml/report'}),
    'dcat': Prefix({'prefix_prefix': 'dcat', 'prefix_reference': 'http://www.w3.org/ns/dcat#'}),
    'example': Prefix

In [5]:
# Get all classes
classes = schema_view.all_classes()
for class_name, class_def in classes.items():
    print(f"Class: {class_name}, Definition: {class_def}")

# Get all slots
slots = schema_view.all_slots()
for slot_name, slot_def in slots.items():
    print(f"Slot: {slot_name}, Definition: {slot_def}")

# Get all enums
enums = schema_view.all_enums()
for enum_name, enum_def in enums.items():
    print(f"Enum: {enum_name}, Definition: {enum_def}")


Class: NamedThing, Definition: ClassDefinition({
  'name': 'NamedThing',
  'description': 'A generic grouping for any identifiable entity',
  'from_schema': 'https://w3id.org/bridge2ai/standards-schema',
  'slots': ['id', 'category', 'name', 'description', 'subclass_of', 'related_to',
    'contributor_name', 'contributor_github_name', 'contributor_orcid',
    'contribution_date'],
  'class_uri': 'schema:Thing'
})
Class: AnatomicalEntity, Definition: ClassDefinition({
  'name': 'AnatomicalEntity',
  'id_prefixes': ['uberon'],
  'description': 'A subcellular location, cell type or gross anatomical part',
  'from_schema': 'https://w3id.org/bridge2ai/standards-schema',
  'exact_mappings': ['uberon:0001062', 'wikidata:Q4936952'],
  'narrow_mappings': ['ncit:C12219'],
  'is_a': 'NamedThing'
})
Class: Organization, Definition: ClassDefinition({
  'name': 'Organization',
  'description': ('Represents a group or organization related to or responsible for one or more '
     'Bridge2AI standards.

In [13]:
class_details = {}

for class_name, class_def in schema_view.all_classes().items():
    description = class_def.description if class_def.description else 'No description available'
    subsets = class_def.in_subset if class_def.in_subset else []
    attributes = {}
    for attr_name, attr_def in class_def.attributes.items():
        attributes[attr_name] = attr_def.description if attr_def.description else 'No description available'
    
    class_details[class_name] = {
        'subset': subsets,
        'attributes': attributes,
        'description': description
    }

# Print the resulting dictionary for verification
for class_name, details in class_details.items():
    print(f"Class: {class_name}")
    print(f"  Subsets: {details['subset']}")
    print(f"  Description: {details['description']}")
    print(f"  Attributes:")
    for attr_name, attr_desc in details['attributes'].items():
        print(f"    - {attr_name}: {attr_desc}")
    print()

Class: NamedThing
  Subsets: []
  Description: A generic grouping for any identifiable entity
  Attributes:

Class: AnatomicalEntity
  Subsets: []
  Description: A subcellular location, cell type or gross anatomical part
  Attributes:

Class: Organization
  Subsets: []
  Description: Represents a group or organization related to or responsible for one or more Bridge2AI standards.
  Attributes:

Class: OrganizationContainer
  Subsets: []
  Description: A container for Organizations.
  Attributes:

Class: Information
  Subsets: []
  Description: Grouping for datasets and data files
  Attributes:

Class: FormatDialect
  Subsets: []
  Description: Additional format information for a file
  Attributes:
    - comment_prefix: No description available
    - delimiter: No description available
    - double_quote: No description available
    - header: No description available
    - quote_char: No description available

Class: Person
  Subsets: []
  Description: An individual human being.
  Attr

In [18]:
# Prepare the class details as a formatted string for the prompt
class_details_str = "\n".join(
    f"Class name: {class_name}\n"
    f"  Subsets: {', '.join(details['subset'])}\n"
    f"  Description: {details['description']}\n"
    f"  Attributes:\n" +
    "".join(f"    {attr_name}: {attr_desc}\n" for attr_name, attr_desc in details['attributes'].items())
    for class_name, details in class_details.items()
)


In [6]:
# Read the Markdown file content
with open("./healthsheet.md", 'r', encoding='utf-8') as file:
    markdown_content = file.read()

# Convert the Markdown content to HTML
html_content = markdown.markdown(markdown_content)

print(html_content)

<h1>Healthsheet for AI-READI Dataset</h1>
<p>This healthsheet provides information about the AI-READI dataset in line with a previously established template (Rostamzadeh et al., https://arxiv.org/abs/2202.13028) aimed at facilitating consistent and transparent documentation regarding the creation, use, and maintenance of healthcare datasets for machine learning.</p>
<h2>General information</h2>
<ol>
<li>Provide a 2 sentence summary of this dataset.</li>
</ol>
<p>The Artificial Intelligence Ready and Equitable Atlas for Diabetes Insights (AI-READI) is a dataset consisting of data collected from individuals with and without "Type 2 Diabetes Mellitus (T2DM)" and harmonized across 3 data collection sites. The composition of the dataset was designed with future studies using AI/Machine Learning in mind. This included recruitment sampling procedures aimed at achieving approximately equal distribution of participants across sex, race, and diabetes severity, as well as the design of a data acq

In [7]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'lxml')

# Find all <li> tags and print them
list_items = soup.find_all('li')
for item in list_items:
    print(item.get_text())

Provide a 2 sentence summary of this dataset.
Has the dataset been audited before? If yes, by whom and what are the results?
Does the dataset get released as static versions or is it dynamically updated?
Is this datasheet created for the original version of the dataset? If not, which version of the dataset is this datasheet for?
Are there any datasheets created for any versions of this dataset?
Does the current version/subversion of the dataset come with predefined task(s), labels, and recommended data splits (e.g., for training, development/validation, testing)? If yes, please provide a high-level description of the introduced tasks, data splits, and labeling, and explain the rationale behind them. Please provide the related links and references. If not, is there any resource (website, portal, etc.) to keep track of all defined tasks and/or associated label definitions? (please note that more detailed questions w.r.t labeling is provided in further sections)
If the dataset has multipl

In [59]:
llm_prompt_template = """
You are provided with a Healthsheets entries and a dictionary of schema class details. Your tasks are as follows:
1. Identify if there is a relevant class for each entry based on the schema Class, Subsets, Description, and class Attributes.
2. If there are multiple potential matches, list all of them.
3. If there is no reasonable match, report "No match" in the 'Schema class name' field and leave the next two fields for subset and relevant fields blank.

Here is the Healthsheet entry:

{list_item}

Here are the schema class details:

{class_details}

For the Healthsheets entry, output the following:
- Schema class name
- Schema class subset
- The most relevant fields for the <li> entry

The output format for each match needs to adhere to this specific format:
Healthsheet entry: "<entry_text>"
  - Schema class name: "<class_name>"
  - Schema class subset: "<subset>"
  - Most relevant fields: "<field1>, <field2>"

""" 

In [61]:
llm_results = []

# Loop over Healthsheets entries, fill out the prompt template, and submit to the LLM
for entry in list_items:
    prompt = llm_prompt_template.format(
        list_item=f"- {entry}",
        class_details=class_details_str
    )
    #print(prompt)
    result = query_openai("gpt-4o", "user", prompt)#"openai/gpt-4o"
    print(result)
    llm_results.append(result)

# Output the results
for result in llm_results:
    print(result)
    print("------")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-9ypoHvYfmdZPxAqlTv19CcWkHY0gj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Healthsheet entry: "Provide a 2 sentence summary of this dataset."\n  - Schema class name: "Dataset"\n  - Schema class subset: "<empty>"\n  - Most relevant fields: "purposes, tasks"\n\nNote: The entry seems to request a summary which is directly related to describing the dataset’s purposes and tasks. The "Dataset" class includes fields that can be used for summarizing the dataset, making it the most relevant match.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1724285865, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_3aa7262c27', usage=CompletionUsage(completion_tokens=91, prompt_tokens=4892, total_tokens=4983))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-9ypoJfZPuunSc30bYRBUxrX2qMpt1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```plaintext\nHealthsheet entry: "Has the dataset been audited before? If yes, by whom and what are the results?"\n  - Schema class name: Dataset\n  - Schema class subset: \n  - Most relevant fields: ethical_reviews, data_protection_impacts\n```', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1724285867, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_df84d6bd70', usage=CompletionUsage(completion_tokens=57, prompt_tokens=4900, total_tokens=4957))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

ChatCompletion(id='chatcmpl-9ypumKcvUzNQpyIqYVecs6LjK3Iok', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here is the Healthsheet entry with the relevant schema class details:\n\nHealthsheet entry: "If the dataset relates to people, are there applicable limits on the retention of the data associated with the instances (e.g., were individuals in question told that their data would be retained for a fixed period of time and then deleted)? If so, please describe these limits and explain how they will be enforced."\n  - Schema class name: "RetentionLimits"\n  - Schema class subset: "Maintenance"\n  - Most relevant fields: "description"\n\nSince the entry is specifically asking about retention limits on data associated with people, the relevant schema class is "RetentionLimits" under the "Maintenance" subset. The most relevant field here is "description".\n\nIn this case, there are no other class matches, so only one class is listed.', 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-9ypuo5gk6uiS0gUqU3mGfzY3D33WT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Certainly! Here is the detailed matching for the provided Healthsheets entry:\n\nHealthsheet entry: "Will older versions of the dataset continue to be supported/hosted/maintained? If so, please describe how and for how long. If not, please describe how its obsolescence will be communicated to users."\n  - Schema class name: "VersionAccess"\n  - Schema class subset: "Maintenance"\n  - Most relevant fields: "description"\n\nThe provided Healthsheets entry queries about the maintenance and communication regarding older dataset versions, which most closely aligns with the "VersionAccess" schema class under the "Maintenance" subset.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1724286270, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_df84d6bd70', u

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-9ypuqcYfCndmCrdipvlgx10FHo11O', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Healthsheet entry: "If others want to extend/augment/build on/contribute to the dataset, is there a mechanism for them to do so? If so, please provide a description. Will these contributions be validated/verified? If so, please describe how. If not, why not? Is there a process for communicating/distributing these contributions to other users? If so, please provide a description."\n  - Schema class name: "ExtensionMechanism"\n  - Schema class subset: "Maintenance"\n  - Most relevant fields: "If others want to extend/augment/build on/contribute to the dataset, is there a mechanism for them to do so? Is there a process for communicating/distributing these contributions to dataset consumers?"\n', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1724286272, model='gpt-4o-2024-05-13', object='chat.compl

In [67]:
# Parse the LLM responses and extract the required data
parsed_data = []
for response in llm_results:
    try:
        # Access the content of the first choice in the response
        response_content = response.choices[0].message.content
        
        # Extract required fields from the response content
        try:
            entry_text = response_content.split('Healthsheet entry: "')[1].split('"')[0].strip()
        except IndexError:
            entry_text = "Not Found"

        try:
            class_name = response_content.split('Schema class name: "')[1].split('"')[0].strip()
        except IndexError:
            class_name = "Not Found"

        try:
            subset = response_content.split('Schema class subset: "')[1].split('"')[0].strip()
        except IndexError:
            subset = "Not Found"

        try:
            most_relevant_fields = response_content.split('Most relevant fields: "')[1].split('"')[0].strip()
        except IndexError:
            most_relevant_fields = "Not Found"

        # Append the extracted data to the parsed_data list
        parsed_data.append({
            "Healthsheet entry": entry_text,
            "Schema class name": class_name,
            "Schema class subset": subset,
            "Most relevant fields": most_relevant_fields
        })
    except Exception as e:
        print(f"An error occurred while parsing response: {e}")
        continue

# Create a DataFrame with the extracted data
df = pd.DataFrame(parsed_data, columns=["Healthsheet entry", "Schema class name", "Schema class subset", "Most relevant fields"])

# Output the DataFrame
print(df)

# Optionally, save the DataFrame to a CSV file
df.to_csv('healthsheet_into_datasheet__llm_results.csv', index=False)

                                     Healthsheet entry   Schema class name  \
0        Provide a 2 sentence summary of this dataset.             Dataset   
1    Has the dataset been audited before? If yes, b...           Not Found   
2    Does the dataset get released as static versio...           Not Found   
3    Is this datasheet created for the original ver...           Not Found   
4    Are there any datasheets created for any versi...             Dataset   
..                                                 ...                 ...   
104  Is there an erratum? If so, please provide a l...             Erratum   
105  Will the dataset be updated (e.g., to correct ...          UpdatePlan   
106  If the dataset relates to people, are there ap...     RetentionLimits   
107  Will older versions of the dataset continue to...       VersionAccess   
108  If others want to extend/augment/build on/cont...  ExtensionMechanism   

    Schema class subset                               Most rele

In [68]:
df[df['Schema class name'] != 'Not Found']

,Healthsheet entry,Schema class name,Schema class subset,Most relevant fields
0,Provide a 2 sentence summary of this dataset.,Dataset,<empty>,"purposes, tasks"
4,Are there any datasheets created for any versi...,Dataset,,"version_access, instances"
5,Does the current version/subversion of the dat...,Dataset,,"tasks, subsets, labeling_strategies"
7,For what purpose was the dataset created? Was ...,Dataset,,"purposes, tasks, addressing_gaps"
8,What are the applications that the dataset is ...,Dataset,,"purposes, tasks, addressing_gaps, existing_uses"
...,...,...,...,...
104,"Is there an erratum? If so, please provide a l...",Erratum,Maintenance,"Is there an erratum?, link or other access point"
105,"Will the dataset be updated (e.g., to correct ...",UpdatePlan,Maintenance,description
106,"If the dataset relates to people, are there ap...",RetentionLimits,Maintenance,description
107,Will older versions of the dataset continue to...,VersionAccess,Maintenance,description


In [70]:
df[df['Schema class name'] == 'Not Found']

,Healthsheet entry,Schema class name,Schema class subset,Most relevant fields
1,"Has the dataset been audited before? If yes, b...",Not Found,Not Found,Not Found
2,Does the dataset get released as static versio...,Not Found,Not Found,Not Found
3,Is this datasheet created for the original ver...,Not Found,Not Found,Not Found
6,"If the dataset has multiple versions, and this...",Not Found,Not Found,Not Found
12,What is the distribution of backgrounds and ex...,Not Found,Not Found,Not Found
16,Not Found,Not Found,Not Found,Not Found
17,What data modality does each patient data cons...,Not Found,Not Found,Not Found
28,Which factors in the data might limit the gene...,Not Found,Not Found,Not Found
31,Not Found,Not Found,Not Found,Not Found
36,Missing data that can be directly filed from o...,Not Found,Not Found,Not Found
